In [5]:
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import pandas as pd
import glob
from copy import deepcopy
from scipy.stats import *
from scipy.stats.stats import pearsonr
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
# self_defined modules
from area_mean import *
from binned_cyFunctions5 import *
from read_hs_file import read_var_mod
from read_var_obs import *
from get_LWPCMIP5data import *
from get_LWPCMIP6data import *
from get_OBSLRMdata import *
from fitLRM_cy1 import *
from fitLRM_cy2 import *
from fitLRM_cy4 import *
from useful_func_cy import *
from calc_Radiation_LRM_1 import *
from calc_Radiation_LRM_2 import *


# calc_LRMobs_metric, fitLRMobs(fitLRM_1, 2, 4)
# & Aploting_Sep11

In [2]:
from calc_LRMobs_metric import *
from fitLRMobs import *
from Aploting_Sep11 import *

In [ ]:
valid_range1=[2013, 1, 15]
valid_range2=[2016, 12, 31]
valid_range3=[2003, 1, 15]
valid_range4=[2012, 12, 31]

In [4]:
# -----------------
# 'valid_range1' and 'valid_range2' give the time stamps of starting and ending times of data for training,
# 'valid_range3' and 'valid_range4' give the time stamps of starting and ending times of data for predicting.
# 'THRESHOLD_sst' is the cut-off of 'Sea surface temperature' for partitioning the 'Hot'/'Cold' LRM regimes;
# 'THRESHOLD_sub' is the cut-off of '500 mb Vertical Velocity (Pressure)' for partitioning 'Up'/'Down' regimes.
# ..
# ------------------
# Southern Ocean 5 * 5 degree bin box
# Using to do area_mean
s_range = arange(-90., 90., 5.) + 2.5  #..global-region latitude edge: (36)
x_range = arange(-180., 180., 5.)  #..logitude sequences edge: number: 72
y_range = arange(-85, -40., 5.) + 2.5  #..southern-ocaen latitude edge: 9


# Function #1 loopping through variables space to find the cut-offs of LRM (Multi-Linear Regression Model).
dict_training, lats_Array, lons_Array, times_Array_training = Pre_processing(s_range, x_range, y_range, valid_range1 = valid_range1, valid_range2 = valid_range2)

dict_predict, lats_Array, lons_Array, times_Array_predict = Pre_processing(s_range, x_range, y_range, valid_range1 = valid_range3, valid_range2 = valid_range4)

# Loop_OBS_LRM(dict_training, dict_predict, s_range, x_range, y_range)


# Function #2 training LRM with using no cut-off, then use it to predict another historical period.
predict_result_1r = fitLRMobs_1(dict_training, dict_predict, s_range, y_range, x_range, lats_Array, lons_Array)

# plotting:

# monthly, binned

LWP_obs_trends(predict_result_1r['LWP_actual_predict'], predict_result_1r['LWP_predi_predict'], predict_result_1r['LWP_actual_training'], predict_result_1r['LWP_predi_training'], times_Array_predict, times_Array_training, y_range, x_range, data_type = '3', running_mean_window = 2)

print(pearsonr(area_mean(annually_mean(predict_result_1r['LWP_actual_predict'], times_Array_predict, label = 'mon'), y_range, x_range), area_mean(annually_mean(predict_result_1r['LWP_predi_predict'], times_Array_predict, label = 'mon'), y_range, x_range))

SyntaxError: invalid syntax (<ipython-input-4-7837f7ddc117>, line 30)

In [3]:
# Function #3,4,5 training LRM with cut_off (TR_sst &/or TR_sub)

WD = '/glade/scratch/chuyan/obs_output/'
folder = glob.glob(WD + 'OBS__' + 'STAT_pi+abr_'+'22x_31y_Sep11th'+ '.npz')
print('cut-off folder', folder)

output_ARRAY = np.load(folder[0], allow_pickle=True)  # str(TR_sst)

TR_sst1 = output_ARRAY['TR_minabias_SST']
TR_sub1 = output_ARRAY['TR_minabias_SUB']
TR_sst2 = output_ARRAY['TR_maxR2_SST']
TR_sub2 = output_ARRAY['TR_maxR2_SUB']

print("TR_min_abs(bias): " , TR_sst1, '  K ', TR_sub1 , ' Pa/s ')
print("TR_large_pi_R_2: ", TR_sst2, '  K ', TR_sub2 , ' Pa/s ')

predict_result_2r_updown = fitLRMobs_2_updown(dict_training, dict_predict, TR_sst2, TR_sub2, s_range, y_range, x_range, lats_Array, lons_Array)

predict_result_2r_hotcold = fitLRMobs_2_hotcold(dict_training, dict_predict, TR_sst2, TR_sub2, s_range, y_range, x_range, lats_Array, lons_Array)

predict_result_4r = fitLRMobs_4(dict_training, dict_predict, TR_sst2, TR_sub2, s_range, y_range, x_range, lats_Array, lons_Array)

# Plotting for the 'Liquid Water Path' trends in the historical period
LWP_obs_trends(predict_result_2r_updown['LWP_actual_predict'], predict_result_2r_updown['LWP_predi_predict'], predict_result_2r_updown['LWP_actual_training'], predict_result_2r_updown['LWP_predi_training'], times_Array_predict, times_Array_training, y_range, x_range, data_type = '3', running_mean_window = 2)

LWP_obs_trends(predict_result_2r_hotcold['LWP_actual_predict'], predict_result_2r_hotcold['LWP_predi_predict'], predict_result_2r_hotcold['LWP_actual_training'], predict_result_2r_hotcold['LWP_predi_training'], times_Array_predict, times_Array_training, y_range, x_range, data_type = '3', running_mean_window = 2)

LWP_obs_trends(predict_result_4r['LWP_actual_predict'], predict_result_4r['LWP_predi_predict'], predict_result_4r['LWP_actual_training'], predict_result_4r['LWP_predi_training'], times_Array_predict, times_Array_training, y_range, x_range, data_type = '3', running_mean_window = 2)


cut-off folder ['/glade/scratch/chuyan/obs_output/OBS__STAT_pi+abr_22x_31y_Sep11th.npz']
TR_min_abs(bias):  284.5258187711201   K  -0.014617270429645265  Pa/s 
TR_large_pi_R_2:  283.9439510076129   K  0.004837945345928386  Pa/s 


NameError: name 'fitLRMobs_2_updown' is not defined

In [ ]:
# CCF trends in historical period:

LWP_obs_trends_2(dict_predict['p_e'], dict_predict['p_e'], times_Array_predict, y_range, x_range, data_type = '1', running_mean_window = 2)

LWP_obs_trends(dict_predict['SUB'], dict_predict['SUB'], dict_training['SUB'], dict_training['SUB'], times_Array_predict, times_Array_training, y_range, x_range, data_type = '3', running_mean_window = 2)